In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import numpy as np
import os

In [ ]:
################################# TRAIN #################################
import config
root_dir = config.DATASET_DIR
#folder = '1000-1500_dullrazor'
folder = '4000-6000_raw'
labels = [1,2,3,4,0]
oimpath = []
y = []
impath = []
for label in labels:
    for name in os.listdir(os.path.join(root_dir,folder,str(label))):
        oimpath.append(os.path.join(root_dir,folder,str(label),name))
        impath.append(os.path.join(root_dir,str(label),name))
    y = oimpath + [label]*len(os.listdir(os.path.join(root_dir,folder,str(label))))

In [ ]:
################################# TEST #################################
import config
root_dir = config.DATASET_DIR
#folder = '1000-1500_dullrazor'
folder = '4000-6000_raw'
oimpath = []
y = []
impath = []
names = os.listdir(os.path.join(root_dir,'Test',folder))
for label in labels:
    for name in os.listdir(os.path.join(root_dir,folder)):
        oimpath.append(os.path.join(root_dir,folder,str(label),name))
        impath.append(os.path.join(root_dir,str(label),name))
    y = oimpath + [label]*len(os.listdir(os.path.join(root_dir,folder,str(label))))

In [ ]:
import pandas as pd
file1 = 'C:\\Users\\mcastro\\Documents\\MCastro\\2_Codigo\\0_DATASETS\\HAM10000_todo' + '\\' + 'ISIC2018_Task3_Test_NatureMedicine_AI_Interaction_Benefit.csv'
metadata_1 = pd.read_csv(file1,index_col=False)
metadata_1.head()

In [ ]:
file2 = 'C:\\Users\\mcastro\\Documents\\MCastro\\2_Codigo\\0_DATASETS\\HAM10000_todo' + '\\' + 'HAM10000_metadata.csv'
metadata_2 = pd.read_csv(file2,index_col=False).fillna(0)
metadata_2.info()

In [ ]:
import numpy as np
diagnosis, diagnosis_n_samples = np.unique(metadata_2['dx'], return_counts=True)
ratio = diagnosis_n_samples / diagnosis_n_samples[4]
print(diagnosis,diagnosis[4])
print(diagnosis_n_samples)
print(ratio)

In [ ]:
from config import DATASET_DIR
print(DATASET_DIR)

In [ ]:
from PIL import Image
ifile = 'C:\\Users\\mcastro\\Documents\\MCastro\\2_Codigo\\0_DATASETS\\HAM10000_todo\\HAM10000_images\\' + 'ISIC_0024306.jpg'
x = Image.open(ifile).convert('RGB')
print(x.size)

In [ ]:
print(len(oimpath),len(impath),oimpath[0],impath[0])

# Preprocessing
* Dataset: 4 bigger classes in ISIC2018 (around 31000 images in those)
* Aim: Hair removal and resizing from 4000 * 6000 to 128 * 128
* A few copies of the "original" dataset are made to 128 * 128 size with and without hairs. 

In [ ]:
import cv2

# https://www.kaggle.com/antocad/siim-isic-image-preprocessing-dull-razor
def dullrazor(img, lowbound=15, filterstruc=3, inpaintmat=3, showimgs=True):
    #grayscale
    imgtmp1 = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #applying a blackhat
    filterSize = (filterstruc, filterstruc)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize) 
    imgtmp2 = cv2.morphologyEx(imgtmp1, cv2.MORPH_BLACKHAT, kernel)
    #0=skin and 255=hair
    ret, mask = cv2.threshold(imgtmp2, lowbound, 255, cv2.THRESH_BINARY)
    #inpainting
    img_final = cv2.inpaint(img, mask, inpaintmat ,cv2.INPAINT_TELEA)

    if showimgs:
        print("_____DULLRAZOR_____")
        plt.imshow(img)
        plt.show()
        plt.imshow(img_final)
        plt.show()
        plt.imshow(imgtmp1, cmap="gray")
        plt.show()
        plt.imshow(imgtmp2, cmap='gray')
        plt.show()
        plt.imshow(mask, cmap='gray')
        plt.show()
    return img_final

In [ ]:
# PRUEBAS PREVIAS

import matplotlib.pyplot as plt
from torchvision.transforms.functional import resize

from PIL import Image

#idx = 31252+17
idx = 16+1+1
#image_with_hair = Image.open(oimpath[idx])
image_with_hair = Image.open(oimpath[idx]).convert('RGB')

#size_0 = [512,768]
size_0 = [1000,1500]
# size_0 = [2000,3000] # No viable. Olvidate.
image_with_hair = resize(image_with_hair,size_0)

image_nohair = dullrazor(np.array(image_with_hair),lowbound=20, filterstruc=16, inpaintmat=7, showimgs=True)
final_image = Image.fromarray(image_nohair)

#size = [128,128]
#final_image = resize(Image.fromarray(image_nohair),size)

final_image.save(impath[idx])

#plt.imshow(final_image)
#plt.show()

In [ ]:
# GUARDAR IMAGENES CON LOS PARAMS ESTIMADOS ANTES
from PIL import Image
from torchvision.transforms.functional import resize

#size_0 = [512,768]
size_0 = [1000,1500]

if size_0 == [512,768]:
    lowbound = 20
    filterstruc = 6
    inpaintmat = 7
elif size_0 == [1000,1500]:
    lowbound = 20
    filterstruc = 16
    inpaintmat = 5

size = [512,768]
#size = [128,128]

for i in range(len(oimpath)):
    image = Image.open(oimpath[i]).convert('RGB')
#    image = resize(image,size_0)
#    image = dullrazor(np.array(image),lowbound=lowbound, filterstruc=filterstruc, inpaintmat=inpaintmat, showimgs=False)
#    new_image = Image.fromarray(image)
    new_image = resize(image,size)
    new_image.save(impath[i])

# Data augmentation

In [1]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import numpy as np
import os
import config

In [2]:
import Augmentor
root_dir = config.DATASET_DIR
augment_dir = os.path.join(root_dir,'ISIC_2020','Train','4000-6000_raw','1')
p = Augmentor.Pipeline(augment_dir)

Initialised with 547 image(s) found.
Output directory set to C:\Users\mcastro\Documents\MCastro\2_Codigo\0_DATASETS\ISIC_2020\Train\4000-6000_raw\1\output.

In [3]:
# Defining pipeline
p.rotate90(probability=0.5)
p.rotate270(probability=0.5)
p.flip_left_right(probability=0.5)
p.flip_top_bottom(probability=0.5)
p.resize(probability=1.0, width=768, height=512)

In [4]:
p.sample(548)
#images, labels = p.sample(548)


Processing <PIL.Image.Image image mode=RGB size=768x512 at 0x260AD55E1C0>: 100%|██████████| 548/548 [00:11<00:00, 49.15 Samples/s]
